# **Workshop #2**

### *Data Pipeline - `spotify` and `the_grammy_awards` dataset*
---

This notebook documents the ETL process for integrating `spotify` and `the_grammy_awards` datasets. The workflow includes:

1. **Setting the Project Directory**: Ensuring the correct working directory.

2. **Importing Dependencies**: Loading necessary modules.

3. **Extracting Data**:

    - **Spotify**: From a CSV file.
    - **Grammys**: From a PostgreSQL database.

4. **Transforming Data**: Cleaning and transforming both datasets.

5. **Merging Data**: Combining the cleaned datasets.

6. **Loading Data**: Into a database.

7. **Storing Data**: Uploading as a CSV to Google Drive.

## ***Setting the project directory***
This script attempts to change the current working directory to the specified path.
If the directory change fails due to the directory not being found, it prints a message indicating that the user is already in the correct directory.

In [1]:
import os

try:
    os.chdir("../../etl-workshop-2/src")
except FileNotFoundError:
    print("You are already in the correct directory.")

## ***Importing dependencies***

**Modules for extraction:**
* **src.extract.spotify_extract**
* **src.extract.grammys_extract**: uses `src.database.db_operations`

**Modules for transformation:**
* **src.transform.spotify_transform**
* **src.transform.grammys_transform**

**Modules for merge:**
* **src.transform.merge**

**Modules for load:**
* **src.load_and_store.load**

**Modules for storing:**
* **src.load_and_store.store**

---

**For this environment we are using:**
* ***Pandas*** >= 2.2.2

**From the `src.database.db_operations` module, we are also using:**
* ***SQLAlchemy*** >= 2.0.32
    * *SQLAlchemy Utils* >= 0.41.2
* ***python-dotenv*** >= 1.0.1

In [2]:
from extract.spotify_extract import extracting_spotify_data
from extract.grammys_extract import extracting_grammys_data

from transform.spotify_transform import transforming_spotify_data
from transform.grammys_transform import transforming_grammys_data
from transform.merge import merging_datasets

from load_and_store.load import loading_merged_data
from load_and_store.store import storing_merged_data

import pandas as pd

# import matplotlib.pyplot as plt
# import seaborn as sns
# plt.style.use("ggplot")

## ***Extracting the data***
---

### **Spotify dataset**
In this section we extract the CSV using the `spotify_extract` module functions. With the use of these functions we can further synthesize our ETL process, and they will be very useful for when we create the tasks using Apache Airflow.

In [3]:
spotify_data = extracting_spotify_data("../data/spotify_dataset.csv")

23/09/2024 09:11:49 PM Data extracted from ../data/spotify_dataset.csv.


In [4]:
spotify_data.head()

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,...,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,...,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,...,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,...,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


### **Grammys dataset**

The extraction process from the PostgreSQL database is performed from the `grammys_extract` module, facilitating the generation of logs in our ETL process. There is no need to create or dispose the connection engine from the notebook, as this process is already done in the module.

In [5]:
grammys_data = extracting_grammys_data()

23/09/2024 09:11:52 PM Engine created. You can now connect to the database.
23/09/2024 09:11:52 PM Extracting data from the Grammy Awards table.
23/09/2024 09:11:56 PM Data extracted from the Grammy Awards table.
23/09/2024 09:11:56 PM Engine disposed.


In [6]:
grammys_data.head()

,year,title,published_at,updated_at,category,nominee,artist,workers,img,winner
0,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,Bad Guy,Billie Eilish,"Finneas O'Connell, producer; Rob Kinelski & Fi...",https://www.grammy.com/sites/com/files/styles/...,True
1,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,"Hey, Ma",Bon Iver,"BJ Burton, Brad Cook, Chris Messina & Justin V...",https://www.grammy.com/sites/com/files/styles/...,True
2,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,7 rings,Ariana Grande,"Charles Anderson, Tommy Brown, Michael Foster ...",https://www.grammy.com/sites/com/files/styles/...,True
3,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,Hard Place,H.E.R.,"Rodney “Darkchild” Jerkins, producer; Joseph H...",https://www.grammy.com/sites/com/files/styles/...,True
4,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,Talk,Khalid,"Disclosure & Denis Kosiak, producers; Ingmar C...",https://www.grammy.com/sites/com/files/styles/...,True


## ***Transforming the data***
---

### *Spotify transformations*
  
- Created a `transforming_spotify_data` function to clean and transform the Spotify DataFrame by:

  - Removing unnecessary columns (e.g., `"Unnamed: 0"`).

  - Eliminating null values and resetting the DataFrame index.

  - Removing duplicates through several steps:
    - Dropped exact duplicate rows.
    - Removed duplicates based on the `"track_id"` column.
    - Mapped detailed genres to broader categories using a predefined genre mapping dictionary.
    - Dropped duplicates based on song names and artists, keeping the most popular entries.

  - Generated new columns for enhanced data analysis:
    - **`duration_min`**: Converted song duration from milliseconds to minutes.
    - **`duration_category`**: Categorized songs based on their duration.
    - **`popularity_category`**: Categorized songs based on their popularity scores.
    - **`track_mood`**: Identified the mood of songs using valence scores.
    - **`live_performance`**: Flagged songs with a high likelihood of being live performances.

  - Dropped irrelevant columns to streamline the dataset (e.g., `"loudness"`, `"mode"`, `"tempo"`).
  
  - Included logging statements to document the cleaning and transformation process, as well as to catch any potential errors.

In [7]:
spotify_df = transforming_spotify_data(spotify_data)

23/09/2024 09:11:57 PM Cleaning and transforming the DataFrame. You currently have 114000 rows and 21 columns.
23/09/2024 09:11:57 PM The dataframe has been cleaned and transformed. You are left with 81343 rows and 14 columns.


In [8]:
spotify_df.head()

,track_id,artists,album_name,track_name,popularity,explicit,danceability,energy,track_genre,duration_min,duration_category,popularity_category,track_mood,live_performance
0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,False,0.676,0.4610,Instrumental,3,Average,High Popularity,Happy,False
1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,False,0.420,0.1660,Instrumental,2,Short,Average Popularity,Sad,False
2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,False,0.438,0.3590,Instrumental,3,Average,Average Popularity,Sad,False
3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,False,0.266,0.0596,Instrumental,3,Average,High Popularity,Sad,False
4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,False,0.618,0.4430,Instrumental,3,Average,High Popularity,Sad,False


### *Grammys Transformations*

- Created a `transforming_grammys_data` function to clean and transform the Grammy Awards DataFrame by:

  - Renaming the column `winner` to `is_nominated`.

  - Dropping unnecessary columns (e.g., `published_at`, `updated_at`, `img`).

  - Removing rows with null values in `nominee`.

  - Handling cases where both `artist` and `workers` are null:
    - Filtered out specific categories listed in the `categories` list.
    - For the remaining rows, filled `artist` with the value from `nominee`.

  - Populating the `artist` column by applying several functions:
    - **`extract_artist`**: Extracted artist names within parentheses from the `workers` column.
    - **`move_workers_to_artist`**: Moved data from `workers` to `artist` if `artist` is null and `workers` doesn't contain semicolons or commas.
    - **`extract_artists_before_semicolon`**: Extracted artist names before semicolons in `workers`, excluding any roles of interest.
    - **`extract_roles_based_on_interest`**: Extracted names associated with specific roles defined in the `roles_of_interest` list from `workers`.

  - Dropped rows with null values in `artist`.

  - Replaced certain values in the `artist` column (e.g., changing `(Various Artists)` to `Various Artists`).

  - Dropped the `workers` column as it was no longer needed.

  - Included logging statements to document the cleaning and transformation process, as well as to catch any potential errors.

In [9]:
grammys_df = transforming_grammys_data(grammys_data)

23/09/2024 09:11:57 PM Starting transformation. The DataFrame has 4810 rows and 10 columns.
23/09/2024 09:11:57 PM Transformation complete. The DataFrame now has 4771 rows and 6 columns.


In [10]:
grammys_df.head()

,year,title,category,nominee,artist,is_nominated
0,2019,62nd Annual GRAMMY Awards (2019),Record Of The Year,Bad Guy,Billie Eilish,True
1,2019,62nd Annual GRAMMY Awards (2019),Record Of The Year,"Hey, Ma",Bon Iver,True
2,2019,62nd Annual GRAMMY Awards (2019),Record Of The Year,7 rings,Ariana Grande,True
3,2019,62nd Annual GRAMMY Awards (2019),Record Of The Year,Hard Place,H.E.R.,True
4,2019,62nd Annual GRAMMY Awards (2019),Record Of The Year,Talk,Khalid,True


## ***Merging the data***
---

- Created the `merging_datasets` function to combine the Spotify and Grammy Awards DataFrames by:

  - **Cleaning key columns for accurate merging**:
    - Converted the `track_name` column in the Spotify DataFrame to lowercase and stripped whitespace, creating a new column `track_name_clean`.
    - Converted the `nominee` column in the Grammys DataFrame to lowercase and stripped whitespace, creating a new column `nominee_clean`.

  - **Merging the datasets**:
    - Performed a left join on the cleaned columns `track_name_clean` and `nominee_clean` to merge the DataFrames.
    - Used suffixes to differentiate overlapping columns, appending `_grammys` to columns from the Grammys DataFrame when necessary.

  - **Handling missing values**:
    - Filled null values in the `title` and `category` columns with `"Not applicable"`.
    - Filled null values in the `is_nominated` column with `False`.

  - **Dropping unnecessary columns**:
    - Removed columns that were no longer needed after the merge, such as `"year"`, `"artist"`, `"nominee"`, `"nominee_clean"`, and `"track_name_clean"`.

- Included helper functions to streamline the data processing:

  - **`fill_null_values`**: Filled null values in specified columns with a given value to ensure data completeness.

  - **`drop_columns`**: Dropped specified columns from the DataFrame to eliminate redundancy and maintain a clean dataset.

- **Used logging throughout** the process to monitor the merging steps and catch any potential issues, enhancing traceability and debugging capability.

In [11]:
merged_df = merging_datasets(spotify_df, grammys_df)

23/09/2024 09:11:57 PM Starting dataset merge.
23/09/2024 09:11:57 PM Initial Spotify dataset has 81343 rows and 14 columns.
23/09/2024 09:11:57 PM Initial Grammys dataset has 4771 rows and 6 columns.
23/09/2024 09:11:58 PM Merge process completed. The final dataframe has 81941 rows and 18 columns.


In [12]:
merged_df.query("is_nominated == True").head()

,id,track_id,artists,album_name,track_name,popularity,explicit,danceability,energy,track_genre,duration_min,duration_category,popularity_category,track_mood,live_performance,title,category,is_nominated
6,6,6Vc5wAMmXdKIAM7WUoEb7N,A Great Big World;Christina Aguilera,Is There Anybody Out There?,Say Something,74,False,0.407,0.147,Instrumental,3,Average,High Popularity,Sad,False,57th Annual GRAMMY Awards (2014),Best Pop Duo/Group Performance,True
8,8,0IktbUcnAGrvD03AWnz3Q8,Jason Mraz;Colbie Caillat,We Sing. We Dance. We Steal Things.,Lucky,74,False,0.625,0.414,Instrumental,3,Average,High Popularity,Happy,False,52nd Annual GRAMMY Awards (2009),Best Pop Collaboration With Vocals,True
12,12,4ptDJbJl35d7gQfeNteBwp,Dan Berk,Solo,Solo,52,False,0.489,0.314,Instrumental,3,Average,Average Popularity,Happy,False,44th Annual GRAMMY Awards (2001),Best Rock Gospel Album,True
39,39,4oa14QBfWRDfJy2agySy0L,Sara Bareilles,Little Voice,Gravity,67,False,0.270,0.275,Instrumental,3,Average,Average Popularity,Sad,False,55th Annual GRAMMY Awards (2012),Best Gospel Album,True
40,40,4oa14QBfWRDfJy2agySy0L,Sara Bareilles,Little Voice,Gravity,67,False,0.270,0.275,Instrumental,3,Average,Average Popularity,Sad,False,51st Annual GRAMMY Awards (2008),Best Solo Rock Vocal Performance,True


When performing the merge we can notice a change in the number of rows: an increase of approximately 600 records. Why is this happening?

In [13]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81941 entries, 0 to 81940
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   81941 non-null  int64  
 1   track_id             81941 non-null  object 
 2   artists              81941 non-null  object 
 3   album_name           81941 non-null  object 
 4   track_name           81941 non-null  object 
 5   popularity           81941 non-null  int64  
 6   explicit             81941 non-null  bool   
 7   danceability         81941 non-null  float64
 8   energy               81941 non-null  float64
 9   track_genre          81941 non-null  object 
 10  duration_min         81941 non-null  int64  
 11  duration_category    81941 non-null  object 
 12  popularity_category  81941 non-null  object 
 13  track_mood           81941 non-null  object 
 14  live_performance     81941 non-null  bool   
 15  title                81941 non-null 

When we look at the number of unique values for *track_id* we see that it is still the same as the Spotify dataset. However, this ensures that suddenly the rise in the number of rows is due to an increase in duplicates within our dataframe.

In [14]:
merged_df["track_id"].nunique()

81343

We are going to group the duplicated songs in order to know what could condition this increase in the number of records.

In [15]:
duplicated_counts = (merged_df
                     .groupby(["track_id", "track_name", "artists", "album_name"])
                     .size()
                     .reset_index(name="duplicate_count")
                     .sort_values(by="duplicate_count", ascending=False))

duplicated_values = duplicated_counts.query("duplicate_count > 1")

In [16]:
duplicated_values.head()

,track_id,track_name,artists,album_name,duplicate_count
7160,0gaZi6J3Pk7FG7GNMHsK5o,Bridge Over Troubled Water,Elvis Presley,That's the Way It Is,7
49675,4kd8NJrrfzzPvarSre2yAu,Need You Now,Armin van Buuren;Jake Reese,Need You Now,5
47179,4W5lqrBjMNk8kNCOIrWv6i,Need You Now,Venemy;DaNica,Need You Now,5
75877,7GAaTpSoTWUTbP2Yxlt4Hd,Need You Now,Lady A,Need You Now,5
66858,6OzRF5Oqmsb1Ec1sMKY4g7,Need You Now,William Black;Forester,Pieces,5


**Looking at the results for this song, we can see that:**

* The fact that a record starts repeating several times may be due to the fact that the song has several nominations.

* **However, the merge criteria can also lead to confusion**: here we find that Elvis is nominated for *Best R&B Vocal Performance, Female*. Elvis is so ubiquitous that he ended up being nominated in a female category.

    * This is caused by the similarity of the values in *track_name*. In order to avoid it, we should add one more criterion to the merge process, but it would imply a much more careful and specific cleanup in the *artist* columns of both datasets..

In [17]:
merged_df.query("track_id == '0gaZi6J3Pk7FG7GNMHsK5o'").head()

,id,track_id,artists,album_name,track_name,popularity,explicit,danceability,energy,track_genre,duration_min,duration_category,popularity_category,track_mood,live_performance,title,category,is_nominated
65918,65918,0gaZi6J3Pk7FG7GNMHsK5o,Elvis Presley,That's the Way It Is,Bridge Over Troubled Water,59,False,0.21,0.236,Rock/Metal,4,Average,Average Popularity,Sad,False,14th Annual GRAMMY Awards (1971),"Best R&B Vocal Performance, Female",True
65919,65919,0gaZi6J3Pk7FG7GNMHsK5o,Elvis Presley,That's the Way It Is,Bridge Over Troubled Water,59,False,0.21,0.236,Rock/Metal,4,Average,Average Popularity,Sad,False,13th Annual GRAMMY Awards (1970),Record Of The Year,True
65920,65920,0gaZi6J3Pk7FG7GNMHsK5o,Elvis Presley,That's the Way It Is,Bridge Over Troubled Water,59,False,0.21,0.236,Rock/Metal,4,Average,Average Popularity,Sad,False,13th Annual GRAMMY Awards (1970),Album Of The Year,True
65921,65921,0gaZi6J3Pk7FG7GNMHsK5o,Elvis Presley,That's the Way It Is,Bridge Over Troubled Water,59,False,0.21,0.236,Rock/Metal,4,Average,Average Popularity,Sad,False,13th Annual GRAMMY Awards (1970),Song Of The Year,True
65922,65922,0gaZi6J3Pk7FG7GNMHsK5o,Elvis Presley,That's the Way It Is,Bridge Over Troubled Water,59,False,0.21,0.236,Rock/Metal,4,Average,Average Popularity,Sad,False,13th Annual GRAMMY Awards (1970),Best Arrangement Accompanying Vocalist(s),True


## ***Loading the data***
---

The `loading_clean_data` function loads a Pandas DataFrame into a database. It logs the process, creates an engine, and tries to load data using `load_clean_data`. If successful, it logs a success message, otherwise logs any errors. The `load_clean_data` function checks if the table exists, creates it if needed, and loads the data.

In [18]:
loading_merged_data(merged_df, "merged_data")

23/09/2024 09:11:58 PM Loading clean data to the database.
23/09/2024 09:12:01 PM Engine created. You can now connect to the database.
23/09/2024 09:12:01 PM Creating table merged_data from Pandas DataFrame.
23/09/2024 09:12:03 PM Adjusting column artists to Text due to length 513.
23/09/2024 09:12:03 PM Adjusting column track_name to Text due to length 511.
23/09/2024 09:12:04 PM Table merged_data created successfully.
23/09/2024 09:12:55 PM Data loaded to table merged_data.
23/09/2024 09:12:55 PM Engine disposed.


## ***Storing the data***
---

The `storing_merged_data` function uploads a merged DataFrame as a CSV file to Google Drive. It authenticates the drive, converts the DataFrame to CSV format, and creates a new file in the specified Google Drive folder. The content is uploaded, and a success message is logged.

In [19]:
storing_merged_data("merged_data", merged_df)

23/09/2024 09:12:55 PM Starting Google Drive authentication process.
23/09/2024 09:12:55 PM access_token is expired. Now: 2024-09-24 02:12:55.993523, token_expiry: 2024-09-20 20:52:25
23/09/2024 09:12:55 PM Access token expired, refreshing token.
23/09/2024 09:12:56 PM Refreshing access_token


23/09/2024 09:12:56 PM Google Drive authentication completed successfully.
23/09/2024 09:12:56 PM Storing merged_data on Google Drive.
23/09/2024 09:13:10 PM File merged_data uploaded successfully.
